# ETL

En esta jupyter notebook se desarrolla la extracción, transformación y carga de los conjuntos de datos.

## Importaciones

In [2]:
import pandas as pd
import json
import gzip
import numpy as np
import ast
from tqdm import tqdm


%load_ext autoreload
%autoreload 2
import utils

import warnings
warnings.filterwarnings("ignore")

## Extracción de los datos

Funcion para extraer los datos ".json" y convertirlos en ".csv" para la exploración.

In [6]:

def procesar_archivo_json(ruta, tipo='json', nombre_archivo_salida='datos.csv'):
    """
    Esta función lee datos desde cada archivo de mi carpeta y los convierte en un DataFrame de pandas.

    Args:
        ruta (str): La ruta al archivo que se va a leer.
        tipo (str, optional): El formato de los datos en el archivo. Puede ser 'json' o 'literal'. 
                              Si es 'json', los datos se cargarán utilizando json.loads. 
                              Si es 'literal', los datos se cargarán utilizando ast.literal_eval. 
                              Por defecto es 'json'.
        nombre_archivo_salida (str, optional): El nombre del archivo CSV de salida.
                                                Por defecto es 'datos.csv'.

    Returns:
        None
    """
    filas = []  # Será la lista para almacenar cada fila de datos

    with gzip.open(ruta, 'rt', encoding='utf-8') as f:
        for line in tqdm(f.readlines(), desc=f"Leyendo {ruta}"):
            if tipo == 'json':
                data = json.loads(line)
            elif tipo == 'literal':
                data = ast.literal_eval(line)
            filas.append(data)

    # Convierte la lista de filas en un DataFrame de pandas
    df = pd.DataFrame(filas)
    
    # Guarda el DataFrame como un archivo CSV
    df.to_csv(nombre_archivo_salida, index=False)

In [8]:
procesar_archivo_json(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\steam_games.json.gz', nombre_archivo_salida='games.csv' )
procesar_archivo_json(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\users_items.json.gz', tipo= 'literal',nombre_archivo_salida='items.csv' )
procesar_archivo_json(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\user_reviews.json.gz', tipo= 'literal', nombre_archivo_salida='reviews.csv')

Leyendo C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\steam_games.json.gz: 100%|██████████| 120445/120445 [00:00<00:00, 136945.55it/s]
Leyendo C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\users_items.json.gz: 100%|██████████| 88310/88310 [02:18<00:00, 636.81it/s] 
Leyendo C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\user_reviews.json.gz: 100%|██████████| 25799/25799 [00:02<00:00, 11262.19it/s]


# Transformación de los datos


### 'games.csv'.


In [18]:
df = pd.read_csv(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\games.csv', encoding = "utf-8")
df.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
cantidad_vacios_por_columna = df.isnull().sum()
cantidad_total_vacios = df.isnull().sum().sum()

print("Cantidad de valores vacíos por columna:")
print(cantidad_vacios_por_columna)
print("\nCantidad total de valores vacíos en el DataFrame:", cantidad_total_vacios)


Cantidad de valores vacíos por columna:
publisher       96381
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

Cantidad total de valores vacíos en el DataFrame: 1169016


In [20]:
df = df.dropna()
cantidad_vacios_por_columna = df.isnull().sum()
cantidad_total_vacios = df.isnull().sum().sum()

print("Cantidad de valores vacíos por columna:")
print(cantidad_vacios_por_columna)
print("\nCantidad total de valores vacíos en el DataFrame:", cantidad_total_vacios)


Cantidad de valores vacíos por columna:
publisher       0
genres          0
app_name        0
title           0
url             0
release_date    0
tags            0
reviews_url     0
specs           0
price           0
early_access    0
id              0
developer       0
dtype: int64

Cantidad total de valores vacíos en el DataFrame: 0


##### Que columnas conservar


Teniendo en cuenta que se nos pide : 

* def *developer*( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.
Mantener : "tags", "developer"

* def *userdata*( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
Mantener : "price"

* def *UserForGenre*( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.
Mantener : "genre", "release_date"

* def *best_developer_year*( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
Mantener : "developer"

* def *developer_reviews_analysis*( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.
Mantener : "developer".

Las columnas a mantener: "developer", "price", "genre", "release_date", "tags", "id", "title". A eliminar: "app_name", "publisher", "url", "early_access", "specs"

In [21]:
del df['app_name']
del df['publisher']
del df['url']
del df['early_access']
del df['specs']

In [22]:
df.columns

Index(['genres', 'title', 'release_date', 'tags', 'reviews_url', 'price', 'id',
       'developer'],
      dtype='object')

In [23]:
df.head()

,genres,title,release_date,tags,reviews_url,price,id,developer
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,4.99,761140.0,Kotoshiro
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,Free To Play,643980.0,Secret Level SRL
88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,Free to Play,670290.0,Poolians.com
88313,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,0.99,767400.0,彼岸领域
88315,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-01-04,"['Action', 'Adventure', 'Simulation', 'FPS', '...",http://steamcommunity.com/app/772540/reviews/?...,3.99,772540.0,Trickjump Games Ltd


In [25]:
df.to_csv(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\games.csv')

### 'items.csv

In [26]:
df = pd.read_csv(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\items.csv')
df.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


### 'reviews.csv'

In [27]:
df = pd.read_csv(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\reviews.csv')
df.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
